In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm
from collections import Counter, defaultdict
import pickle
import csv
import argparse

In [2]:
gdelt_raw = pd.read_csv('../../data_final/GDELT_EGIRIS.csv', sep='\t')

In [3]:
gdelt_raw

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,date,timid,Md5_list,year,md5
0,EGYPT,POLITICAL PARTIES,51,EG,EG,EG,20150219,0,77e46312cec9426b36809390e064f845,2015,77e46312cec9426b36809390e064f845
1,RUSSIAN,RUSSIAN,42,EG,LY,EG,20150219,0,c7887de81c936f036c2c6bfad834f759,2015,c7887de81c936f036c2c6bfad834f759
2,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,b8643082d868be96337d337108616629,2015,b8643082d868be96337d337108616629
3,WEBSITE,EGYPT,42,EG,EG,EG,20150219,0,e9fdf0f07df44291c95f0c377b627ddf,2015,e9fdf0f07df44291c95f0c377b627ddf
4,RUSSIAN,RUSSIAN,43,RS,RS,EG,20150219,0,c7887de81c936f036c2c6bfad834f759,2015,c7887de81c936f036c2c6bfad834f759
...,...,...,...,...,...,...,...,...,...,...,...
3009530,INVESTOR,ISRAEL,43,IS,IS,IS,20220317,2583,a524f2d4fa0df8553456c76c1ba76307,2022,a524f2d4fa0df8553456c76c1ba76307
3009531,GOVERNMENT,ISRAEL,161,IS,IS,IS,20220317,2583,a66b9271e051ec3d20a286685581d3cd,2022,a66b9271e051ec3d20a286685581d3cd
3009532,ISRAEL,ISRAELI,190,IS,IS,IS,20220317,2583,60fdcf21b0493004c019292180f1db19,2022,60fdcf21b0493004c019292180f1db19
3009533,UKRAINIAN,TEL AVIV,20,UP,IS,IS,20220317,2583,bd8afe7c84e4db01b9c39529e6b24b66,2022,bd8afe7c84e4db01b9c39529e6b24b66


In [4]:
gdelt_raw = gdelt_raw.drop('Md5_list', axis=1)

In [6]:
gdelt_raw = gdelt_raw.drop_duplicates(keep='first')

In [7]:
gdelt_raw

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,date,timid,year,md5
0,EGYPT,POLITICAL PARTIES,51,EG,EG,EG,20150219,0,2015,77e46312cec9426b36809390e064f845
1,RUSSIAN,RUSSIAN,42,EG,LY,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759
2,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629
3,WEBSITE,EGYPT,42,EG,EG,EG,20150219,0,2015,e9fdf0f07df44291c95f0c377b627ddf
4,RUSSIAN,RUSSIAN,43,RS,RS,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759
...,...,...,...,...,...,...,...,...,...,...
3009530,INVESTOR,ISRAEL,43,IS,IS,IS,20220317,2583,2022,a524f2d4fa0df8553456c76c1ba76307
3009531,GOVERNMENT,ISRAEL,161,IS,IS,IS,20220317,2583,2022,a66b9271e051ec3d20a286685581d3cd
3009532,ISRAEL,ISRAELI,190,IS,IS,IS,20220317,2583,2022,60fdcf21b0493004c019292180f1db19
3009533,UKRAINIAN,TEL AVIV,20,UP,IS,IS,20220317,2583,2022,bd8afe7c84e4db01b9c39529e6b24b66


In [10]:
gdelt_raw.to_csv(path_or_buf='../../data_final/GDELT_EGIRIS.csv', sep='\t', index=False)

In [11]:
relation_ids = json.load(open('../../data/relation_ids.json', 'r'))

In [13]:
level1s, level2s, level3s = relation_ids
level1s_int = [int(x) for x in level1s]
level2s_int = [int(x) for x in level2s]
level3s_int = [int(x) for x in level3s]

In [14]:
print(len(gdelt_raw[gdelt_raw['EventCode'].isin(level1s_int)]))
print(len(gdelt_raw[gdelt_raw['EventCode'].isin(level2s_int)]))
print(len(gdelt_raw[gdelt_raw['EventCode'].isin(level3s_int)]))

1169379
1725365
114781


In [18]:
print(len(gdelt_raw['Actor1Name']._append(gdelt_raw['Actor2Name']).unique()))
print(len(gdelt_raw['EventCode'].unique()))

4266
244


In [9]:
len(gdelt_raw['md5'].unique())

586690

In [8]:
final_md5_list = json.load(open('./final_md5_list.json', 'r'))

In [20]:
gdelt_ce = gdelt_raw[gdelt_raw['md5'].isin(final_md5_list)]

In [21]:
gdelt_ce

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,date,timid,year,md5
1,RUSSIAN,RUSSIAN,42,EG,LY,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759
2,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629
4,RUSSIAN,RUSSIAN,43,RS,RS,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759
5,EGYPT,SYRIA,195,EG,SY,EG,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a
6,EGYPT,SYRIA,195,EG,EG,JO,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a
...,...,...,...,...,...,...,...,...,...,...
3009524,THE EU,WEST BANK,10,IS,WE,IS,20220317,2583,2022,303902559b8543d41b0d2f3ad2b73569
3009527,ISRAEL,PALESTINIAN,46,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451
3009528,PALESTINIAN,PALESTINE,10,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451
3009531,GOVERNMENT,ISRAEL,161,IS,IS,IS,20220317,2583,2022,a66b9271e051ec3d20a286685581d3cd


In [41]:
md52rawtopic = json.load(open('./md52rawtopic.json', 'r'))

In [42]:
gdelt_ce['Topic'] = [md52rawtopic[_] for _ in gdelt_ce['md5']]

/tmp/ipykernel_2984193/1509153667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdelt_ce['Topic'] = [md52rawtopic[_] for _ in gdelt_ce['md5']]


In [43]:
gdelt_ce

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,date,timid,year,md5,Topic
1,RUSSIAN,RUSSIAN,42,EG,LY,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759,1277
2,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629,-1
4,RUSSIAN,RUSSIAN,43,RS,RS,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759,1277
5,EGYPT,SYRIA,195,EG,SY,EG,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a,-1
6,EGYPT,SYRIA,195,EG,EG,JO,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a,-1
...,...,...,...,...,...,...,...,...,...,...,...
3009524,THE EU,WEST BANK,10,IS,WE,IS,20220317,2583,2022,303902559b8543d41b0d2f3ad2b73569,690
3009527,ISRAEL,PALESTINIAN,46,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1
3009528,PALESTINIAN,PALESTINE,10,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1
3009531,GOVERNMENT,ISRAEL,161,IS,IS,IS,20220317,2583,2022,a66b9271e051ec3d20a286685581d3cd,-1


In [94]:
gdelt_ce.to_csv(path_or_buf='./GDELT_raw.csv', sep='\t', index=False)

In [47]:
print(len(gdelt_ce['md5'].unique()))
print(len(gdelt_ce))
print(len(gdelt_ce['Actor1Name']._append(gdelt_ce['Actor2Name']).unique()))
print(len(gdelt_ce['EventCode'].unique()))

274795
1568393
3338
239


In [ ]:
# filter by entity tfdf

In [48]:
def calculate_frequency(vicuna_dropna):
    # Entity frequency
    term_freq = {k: v for k, v in sorted(Counter(vicuna_dropna['Actor1Name']._append(vicuna_dropna['Actor2Name'])).items(), key=lambda item: item[1], reverse=False)}

    # Document frequency
    df = pd.concat([vicuna_dropna[['Actor1Name', 'md5']], vicuna_dropna[['Actor2Name', 'md5']].rename(columns={'Actor2Name':'Actor1Name'})])
    unique_df = df.drop_duplicates()
    doc_freq = unique_df.groupby('Actor1Name')['md5'].nunique().sort_values(ascending=True).to_dict()

    return term_freq, doc_freq

In [49]:
m = 10
n = 0
data = gdelt_ce.copy()
prev_num_rows = data.shape[0]
while True:
    
    term_freq, doc_freq = calculate_frequency(data)

    mask_tf = data['Actor1Name'].map(term_freq).ge(m) & data['Actor2Name'].map(term_freq).ge(m)
    mask_df = data['Actor1Name'].map(doc_freq).ge(n) & data['Actor2Name'].map(doc_freq).ge(n)

    # Filter raw_data
    data = data[mask_tf & mask_df]

    if data.shape[0] == prev_num_rows:
        entity_counter = Counter(data['Actor1Name']._append(data['Actor2Name']))
        total_entity_filter = {k: v for k, v in sorted(entity_counter.items(), key=lambda item: item[1], reverse=False)}

        md5_counter = Counter(data['md5'])
        total_md5_filter = {k: v for k, v in sorted(md5_counter.items(), key=lambda item: item[1], reverse=False)}

        print(f"filter version {m}_{n}: number of events: {len(data)}")
        print(f"filter version {m}_{n}: number of entities: {len(total_entity_filter)}, number of documents: {len(total_md5_filter)}")
        print()

        break
    else:
        prev_num_rows = data.shape[0]

filter version 10_0: number of events: 1563514
filter version 10_0: number of entities: 1954, number of documents: 274285



In [53]:
gdelt_ce_tf = data

In [51]:
print(len(data['md5'].unique()))
print(len(data))
print(len(data['Actor1Name']._append(data['Actor2Name']).unique()))
print(len(data['EventCode'].unique()))

274285
1563514
1954
239


In [95]:
gdelt_ce_tf['Topic'] = [md52rawtopic[_] for _ in gdelt_ce_tf['md5']]
gdelt_ce_tf.to_csv(path_or_buf='./GDELT_tf.csv', sep='\t', index=False)

In [55]:
m = 10
n = 10
data = gdelt_ce.copy()
prev_num_rows = data.shape[0]
while True:
    
    term_freq, doc_freq = calculate_frequency(data)

    mask_tf = data['Actor1Name'].map(term_freq).ge(m) & data['Actor2Name'].map(term_freq).ge(m)
    mask_df = data['Actor1Name'].map(doc_freq).ge(n) & data['Actor2Name'].map(doc_freq).ge(n)

    # Filter raw_data
    data = data[mask_tf & mask_df]

    if data.shape[0] == prev_num_rows:
        entity_counter = Counter(data['Actor1Name']._append(data['Actor2Name']))
        total_entity_filter = {k: v for k, v in sorted(entity_counter.items(), key=lambda item: item[1], reverse=False)}

        md5_counter = Counter(data['md5'])
        total_md5_filter = {k: v for k, v in sorted(md5_counter.items(), key=lambda item: item[1], reverse=False)}

        print(f"filter version {m}_{n}: number of events: {len(data)}")
        print(f"filter version {m}_{n}: number of entities: {len(total_entity_filter)}, number of documents: {len(total_md5_filter)}")
        print()

        break
    else:
        prev_num_rows = data.shape[0]

filter version 10_10: number of events: 1557862
filter version 10_10: number of entities: 1557, number of documents: 273914



In [56]:
gdelt_ce_tfdf = data

In [57]:
print(len(data['md5'].unique()))
print(len(data))
print(len(data['Actor1Name']._append(data['Actor2Name']).unique()))
print(len(data['EventCode'].unique()))

273914
1557862
1557
239


In [96]:
gdelt_ce_tfdf['Topic'] = [md52rawtopic[_] for _ in gdelt_ce_tfdf['md5']]
gdelt_ce_tfdf.to_csv(path_or_buf='./GDELT_tfdf.csv', sep='\t', index=False)

In [59]:
gdelt_ce_tfdf

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,date,timid,year,md5,Topic
1,RUSSIAN,RUSSIAN,42,EG,LY,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759,1277
2,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629,-1
4,RUSSIAN,RUSSIAN,43,RS,RS,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759,1277
5,EGYPT,SYRIA,195,EG,SY,EG,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a,-1
6,EGYPT,SYRIA,195,EG,EG,JO,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a,-1
...,...,...,...,...,...,...,...,...,...,...,...
3009524,THE EU,WEST BANK,10,IS,WE,IS,20220317,2583,2022,303902559b8543d41b0d2f3ad2b73569,690
3009527,ISRAEL,PALESTINIAN,46,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1
3009528,PALESTINIAN,PALESTINE,10,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1
3009531,GOVERNMENT,ISRAEL,161,IS,IS,IS,20220317,2583,2022,a66b9271e051ec3d20a286685581d3cd,-1


In [60]:
grouplist = list(gdelt_ce_tfdf.groupby(['Actor1Name', 'EventCode', 'Actor2Name', 'Topic', 'timid']))

In [63]:
len(grouplist)

1202297

In [64]:
ce_pds = []
for groupid, group in tqdm(enumerate(grouplist), total=len(grouplist)):
    group_df = group[1]
    md5_list = list(group_df['md5'].unique())
    ce_pd = group_df.head(1).copy()
    ce_pd['Md5_list'] = ', '.join(md5_list)
    ce_pds.append(ce_pd.copy())

100%|███████████████████████████████████████████| 1202297/1202297 [07:26<00:00, 2690.33it/s]


In [65]:
ce_df = pd.concat(ce_pds).sort_index()

In [66]:
ce_df

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,date,timid,year,md5,Topic,Md5_list
1,RUSSIAN,RUSSIAN,42,EG,LY,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759,1277,c7887de81c936f036c2c6bfad834f759
2,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629,-1,"b8643082d868be96337d337108616629, 438806297311..."
4,RUSSIAN,RUSSIAN,43,RS,RS,EG,20150219,0,2015,c7887de81c936f036c2c6bfad834f759,1277,c7887de81c936f036c2c6bfad834f759
5,EGYPT,SYRIA,195,EG,SY,EG,20150219,0,2015,b8c6ea9a1d38101ce4d5f8ea673b404a,-1,b8c6ea9a1d38101ce4d5f8ea673b404a
8,THE UN,ARAB LEAGUE,46,EG,EG,EG,20150219,0,2015,d19f42d1f5e86ea8101e933b2668ed99,-1,d19f42d1f5e86ea8101e933b2668ed99
...,...,...,...,...,...,...,...,...,...,...,...,...
3009524,THE EU,WEST BANK,10,IS,WE,IS,20220317,2583,2022,303902559b8543d41b0d2f3ad2b73569,690,303902559b8543d41b0d2f3ad2b73569
3009527,ISRAEL,PALESTINIAN,46,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1,d52355726f828f51f7deaa065f4fa451
3009528,PALESTINIAN,PALESTINE,10,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1,d52355726f828f51f7deaa065f4fa451
3009531,GOVERNMENT,ISRAEL,161,IS,IS,IS,20220317,2583,2022,a66b9271e051ec3d20a286685581d3cd,-1,a66b9271e051ec3d20a286685581d3cd


In [67]:
print(len(ce_df['md5'].unique()))
print(len(ce_df))
print(len(ce_df['Actor1Name']._append(data['Actor2Name']).unique()))
print(len(ce_df['EventCode'].unique()))

271182
1202297
1557
239


In [69]:
print(len(ce_df[ce_df['Topic']==-1]))

627230


In [70]:
# stats
results = ce_df.groupby(['Topic']).agg({'timid':['count', 'max','min','mean']})
results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
results['nday_range'] = results['nday_max'] -results['nday_min'] + 1
print(results)
print(results.loc[0:, :].mean())
print(results.loc[0:, :].max())
print(results.loc[0:, :].min())

       n_events  nday_max  nday_min    nday_mean  nday_range
Topic                                                       
-1       627230      2583         0  1242.589299        2584
 0         6981      2015      1799  1870.143389         217
 1         3836       410       215   308.316215         196
 2         2186      2272      2070  2169.363678         203
 3         2414       953       825   868.037697         129
...         ...       ...       ...          ...         ...
 3745        48      2474      2444  2458.708333          31
 3746        31      1392      1359  1370.387097          34
 3747        34      1319      1286  1301.205882          34
 3748        19      1228      1215  1217.000000          14
 3749        36       793       781   784.694444          13

[3751 rows x 5 columns]
n_events       153.351200
nday_max      1264.953067
nday_min      1226.774933
nday_mean     1245.200658
nday_range      39.178133
dtype: float64
n_events      6981.000000
nday_max   

In [ ]:
# split by max day range and max cluster size

In [71]:
max_n_range = 78
max_n_events = 306

In [72]:
topicid_max = ce_df['Topic'].max()

In [73]:
topicid_max

3749

In [75]:
ce_pd_splitted = []
for topicid in range(0, topicid_max + 1):
    ce_pd = ce_df[ce_df['Topic']==topicid]
    ce_pd = ce_pd.sort_values(by=['date'], ignore_index=True)
    curr_rowid = 0
    while curr_rowid <= (len(ce_pd)-1):
        next_rowid = min(curr_rowid + max_n_events-1, len(ce_pd)-1)
        next_row = ce_pd.iloc[[next_rowid]]
        next_timid = next_row['timid'].values[0]
        curr_row = ce_pd.iloc[[curr_rowid]]
        curr_timid = curr_row['timid'].values[0]
        n_range = next_timid - curr_timid + 1
        if n_range <= max_n_range: # split by max_n_events
            ce_pd_splitted.append(ce_pd[curr_rowid: next_rowid+1])
            curr_rowid = next_rowid+1
        else: # split by n_range
            next_rowid = len(ce_pd[ce_pd['timid'] <= (curr_timid+max_n_range-1)]) - 1
            ce_pd_splitted.append(ce_pd[curr_rowid: next_rowid+1])
            curr_rowid = next_rowid+1   

In [79]:
new_ces = []
for idx, new_ce in enumerate(ce_pd_splitted):
    new_ce = new_ce.rename(columns={'Topic': 'ce_id', 'date':'day', 'md5': 'Md5'})
    new_ce['ce_id'] = [idx] * len(new_ce)
    new_ces.append(new_ce)

In [77]:
len(new_ces)

4668

In [80]:
new_ces[0]

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,EGYPTIAN,CITIZEN,130,EG,CH,CH,20200123,1799,2020,3d52c8acaab22b919f627ad1fc0d3b70,0,3d52c8acaab22b919f627ad1fc0d3b70
1,POLICE,ISRAEL,20,IS,IS,IS,20200126,1802,2020,ec804b29897268702f74ea6b1de45482,0,ec804b29897268702f74ea6b1de45482
2,ISRAEL,CHINA,43,IS,CH,IS,20200126,1802,2020,ec804b29897268702f74ea6b1de45482,0,ec804b29897268702f74ea6b1de45482
3,ISRAEL,CHINA,173,IS,CH,CH,20200126,1802,2020,ec804b29897268702f74ea6b1de45482,0,ec804b29897268702f74ea6b1de45482
4,ISRAEL,CHINA,112,IS,CH,CH,20200126,1802,2020,ec804b29897268702f74ea6b1de45482,0,ec804b29897268702f74ea6b1de45482
...,...,...,...,...,...,...,...,...,...,...,...,...
301,JAPANESE,ISRAEL,36,JA,IS,IS,20200219,1826,2020,8e1d9c65100c65183fb238926eb3b89b,0,8e1d9c65100c65183fb238926eb3b89b
302,CHINESE,MINISTRY,43,CH,EG,EG,20200219,1826,2020,91ab0eb2f55476777220431a6431414a,0,91ab0eb2f55476777220431a6431414a
303,EGYPT,SCHOOL,43,EG,EG,EG,20200219,1826,2020,91ab0eb2f55476777220431a6431414a,0,91ab0eb2f55476777220431a6431414a
304,SCHOOL,EGYPT,42,EG,EG,EG,20200219,1826,2020,91ab0eb2f55476777220431a6431414a,0,91ab0eb2f55476777220431a6431414a


In [82]:
outliers = ce_df[ce_df['Topic']==-1].sort_values(by=['date'], ignore_index=True)
outliers = outliers.rename(columns={'Topic': 'ce_id', 'date':'day', 'md5': 'Md5'})

In [83]:
outliers

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629,-1,"b8643082d868be96337d337108616629, 438806297311..."
1,US GOVERNMENT,ISRAEL,20,IS,IS,IS,20150219,0,2015,c7583a7778d803b71cacb4b096278b9e,-1,c7583a7778d803b71cacb4b096278b9e
2,ISRAELI,ISRAELI,40,IS,IS,US,20150219,0,2015,a69ea4173461ad41702db21f46fdc320,-1,a69ea4173461ad41702db21f46fdc320
3,JERUSALEM,ISRAEL,20,IS,IS,IS,20150219,0,2015,44d1c4c6a15c91ada8a413dcadc5f19d,-1,44d1c4c6a15c91ada8a413dcadc5f19d
4,TEL AVIV,TEL AVIV,190,IS,IS,IS,20150219,0,2015,1255d42cddaff83ed85ab6a39da8beda,-1,1255d42cddaff83ed85ab6a39da8beda
...,...,...,...,...,...,...,...,...,...,...,...,...
627225,IRANIAN,WASHINGTON,26,IR,US,US,20220317,2583,2022,fdb0cda3fccee4abb4caa2096b9d1005,-1,fdb0cda3fccee4abb4caa2096b9d1005
627226,UNITED KINGDOM,MILITARY,43,IR,UK,UK,20220317,2583,2022,2b79ce5899aa099a03402c9b32888088,-1,2b79ce5899aa099a03402c9b32888088
627227,IRAN,COMMANDER,112,IR,IR,IR,20220317,2583,2022,b8e0699349bda61f9f70af84bfe4238f,-1,b8e0699349bda61f9f70af84bfe4238f
627228,IRAN,CITIZEN,84,MU,IR,UK,20220317,2583,2022,2b79ce5899aa099a03402c9b32888088,-1,2b79ce5899aa099a03402c9b32888088


In [84]:
new_ces.append(outliers)
new_ces_df = pd.concat(new_ces, ignore_index=True)

In [85]:
new_ces_df = new_ces_df.sort_values(by=['day'], ignore_index=True)

In [86]:
new_ces_df

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,JORDAN,EGYPT,40,EG,IS,JO,20150219,0,2015,fea4f048e58509eb5ce3a95a356d3f3f,-1,fea4f048e58509eb5ce3a95a356d3f3f
1,ISRAEL,PRIME MINISTER,11,IS,IS,IS,20150219,0,2015,5a4b044b1fa09e9e5e2c493f80402cab,-1,5a4b044b1fa09e9e5e2c493f80402cab
2,OBAMA,ADMINISTRATION,43,IS,IS,IS,20150219,0,2015,f3d779dfd752412791e48322b5e2d05e,-1,f3d779dfd752412791e48322b5e2d05e
3,GAZA,ISRAEL,160,TU,IS,IS,20150219,0,2015,d6db17beecf1764d75e1b969d6e6e825,-1,d6db17beecf1764d75e1b969d6e6e825
4,GAZA,ISRAEL,161,IS,IS,IS,20150219,0,2015,d6db17beecf1764d75e1b969d6e6e825,-1,d6db17beecf1764d75e1b969d6e6e825
...,...,...,...,...,...,...,...,...,...,...,...,...
1202292,UKRAINE,UKRAINE,110,IS,IS,IS,20220317,2583,2022,cac2d7644682d7109ca0d2d7a5aac8d5,207,cac2d7644682d7109ca0d2d7a5aac8d5
1202293,RUSSIAN,UKRAINIAN,75,IS,IS,IS,20220317,2583,2022,e6dd4bb00d50281de4068a9de2bb236c,207,e6dd4bb00d50281de4068a9de2bb236c
1202294,RUSSIAN,MILITARY BASE,194,IS,IS,IS,20220317,2583,2022,2059ada204307b9c3193f3ba5858d082,207,2059ada204307b9c3193f3ba5858d082
1202295,KYIV,RUSSIA,193,IS,IS,IS,20220317,2583,2022,3f6bef0948c107595279678511ec187d,207,3f6bef0948c107595279678511ec187d


In [87]:
# stats
new_results = new_ces_df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
new_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
new_results['nday_range'] = new_results['nday_max'] -new_results['nday_min'] + 1
print(new_results)
print(new_results.loc[0:, :].mean())
print(new_results.loc[0:, :].max())
print(new_results.loc[0:, :].min())

       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       627230      2583         0  1242.589299        2584
 0          306      1826      1799  1813.640523          28
 1          306      1830      1826  1828.637255           5
 2          306      1832      1830  1830.633987           3
 3          306      1834      1832  1833.032680           3
...         ...       ...       ...          ...         ...
 4663        48      2474      2444  2458.708333          31
 4664        31      1392      1359  1370.387097          34
 4665        34      1319      1286  1301.205882          34
 4666        19      1228      1215  1217.000000          14
 4667        36       793       781   784.694444          13

[4669 rows x 5 columns]
n_events       123.193445
nday_max      1290.913239
nday_min      1261.687661
nday_mean     1276.104833
nday_range      30.225578
dtype: float64
n_events       306.0
nday_max      25

In [ ]:
# filter by min range and min size

In [88]:
unfiltered_ces = new_ces[:-1]

In [89]:
invalid_ces = []
valid_ces = []
for ce in unfiltered_ces:
    if len(ce) < 10:
        invalid_ces.append(ce)
    elif len(ce['timid'].unique()) == 1:
        invalid_ces.append(ce)
    else:
        valid_ces.append(ce)

In [90]:
len(invalid_ces)

132

In [91]:
new_outliers = pd.concat([outliers]+invalid_ces, ignore_index=True)
new_outliers['ce_id'] = [-1] * len(new_outliers)

In [92]:
new_outliers

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,EGYPTIAN,LIBYA,195,EG,EG,EG,20150219,0,2015,b8643082d868be96337d337108616629,-1,"b8643082d868be96337d337108616629, 438806297311..."
1,US GOVERNMENT,ISRAEL,20,IS,IS,IS,20150219,0,2015,c7583a7778d803b71cacb4b096278b9e,-1,c7583a7778d803b71cacb4b096278b9e
2,ISRAELI,ISRAELI,40,IS,IS,US,20150219,0,2015,a69ea4173461ad41702db21f46fdc320,-1,a69ea4173461ad41702db21f46fdc320
3,JERUSALEM,ISRAEL,20,IS,IS,IS,20150219,0,2015,44d1c4c6a15c91ada8a413dcadc5f19d,-1,44d1c4c6a15c91ada8a413dcadc5f19d
4,TEL AVIV,TEL AVIV,190,IS,IS,IS,20150219,0,2015,1255d42cddaff83ed85ab6a39da8beda,-1,1255d42cddaff83ed85ab6a39da8beda
...,...,...,...,...,...,...,...,...,...,...,...,...
628638,SAUDI,ISLAM,51,SA,IR,IR,20171025,979,2017,f73ef51754cb75d14d0957b52af44de1,-1,f73ef51754cb75d14d0957b52af44de1
628639,SAUDI,IRAN,51,AF,IR,IR,20171025,979,2017,f73ef51754cb75d14d0957b52af44de1,-1,f73ef51754cb75d14d0957b52af44de1
628640,TEHRAN,SAUDI ARABIA,186,IR,SA,SA,20171025,979,2017,f73ef51754cb75d14d0957b52af44de1,-1,f73ef51754cb75d14d0957b52af44de1
628641,IRAQ,IRAN,120,IZ,IR,IR,20171025,979,2017,f73ef51754cb75d14d0957b52af44de1,-1,f73ef51754cb75d14d0957b52af44de1


In [93]:
new_outliers_grouplist = list(new_outliers.groupby(['Actor1Name', 'EventCode', 'Actor2Name', 'day']))
print(len(new_outliers_grouplist))
new_outliers_dedu_list = []
for groupid, group in tqdm(enumerate(new_outliers_grouplist), total=len(new_outliers_grouplist)):
    group_df = group[1]
    md5_list = []
    md5_lists = list(group_df['Md5_list'].unique())
    for md5_list_str in md5_lists:
        md5_list += md5_list_str.split(', ')
    kept_outlier = group_df.head(1).copy()
    kept_outlier['Md5_list'] = ', '.join(md5_list)
    new_outliers_dedu_list.append(kept_outlier.copy())
new_outliers_dedu = pd.concat(new_outliers_dedu_list).sort_values(by=['day'], ignore_index=True)

628615


100%|█████████████████████████████████████████████| 628615/628615 [03:57<00:00, 2646.63it/s]


In [97]:
new_outliers_dedu

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,THE WHITE HOUSE,UNITED STATES,110,IS,IS,IR,20150219,0,2015,f3d779dfd752412791e48322b5e2d05e,-1,f3d779dfd752412791e48322b5e2d05e
1,KINGDOM,IRAQ,57,EG,EG,EG,20150219,0,2015,fea4f048e58509eb5ce3a95a356d3f3f,-1,fea4f048e58509eb5ce3a95a356d3f3f
2,THE US,CITIZEN,10,IS,IS,IS,20150219,0,2015,72dcb54c9b49830caefdad4bb7453d3c,-1,72dcb54c9b49830caefdad4bb7453d3c
3,KINGDOM,EGYPT,57,IZ,IZ,EG,20150219,0,2015,fea4f048e58509eb5ce3a95a356d3f3f,-1,fea4f048e58509eb5ce3a95a356d3f3f
4,SPOKESMAN,EGYPT,18,TS,EG,EG,20150219,0,2015,17ec9652bed95915b25ffa18a8a0cf05,-1,17ec9652bed95915b25ffa18a8a0cf05
...,...,...,...,...,...,...,...,...,...,...,...,...
628610,UNITED STATES,AMBASSADOR,46,US,IS,IS,20220317,2583,2022,b407f0de34e9f3a92bb084a67e7aad7c,-1,b407f0de34e9f3a92bb084a67e7aad7c
628611,PALESTINIAN,PALESTINE,10,IS,IS,IS,20220317,2583,2022,d52355726f828f51f7deaa065f4fa451,-1,d52355726f828f51f7deaa065f4fa451
628612,FRENCH,FRANCE,40,UP,IS,UP,20220317,2583,2022,87a046d58f0a05e2c9c1bb9bd826b594,-1,87a046d58f0a05e2c9c1bb9bd826b594
628613,DEFENSE MINIST,IRANIAN,40,IS,IR,IR,20220317,2583,2022,7b4bab0e38720b867a170944cf233c9b,-1,7b4bab0e38720b867a170944cf233c9b


In [98]:
for idx, valid_ce in enumerate(valid_ces):
    valid_ce['ce_id'] = [idx] * len(valid_ce)

In [99]:
filtered_ces = valid_ces + [new_outliers_dedu]
filtered_ces_df = pd.concat(filtered_ces, ignore_index=True)

In [100]:
filtered_ces_df = filtered_ces_df.sort_values(by=['day'], ignore_index=True)

In [101]:
filtered_ces_df

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,PALESTINIAN,ISRAEL,51,IS,IS,IS,20150219,0,2015,18918b73f1e5f11eb8734defabe9a804,-1,18918b73f1e5f11eb8734defabe9a804
1,UNITED KINGDOM,EGYPT,40,UK,EG,EG,20150219,0,2015,edf3db2729f3e81b20e446b94f7a6b6a,-1,edf3db2729f3e81b20e446b94f7a6b6a
2,JERUSALEM,AMBASSADOR,40,EG,IS,IS,20150219,0,2015,40f93517a5da0fd3efa6db594e8d0cb4,-1,40f93517a5da0fd3efa6db594e8d0cb4
3,ARAB LEAGUE,THE UN,46,EG,EG,EG,20150219,0,2015,d19f42d1f5e86ea8101e933b2668ed99,-1,d19f42d1f5e86ea8101e933b2668ed99
4,UNITED STATES,ISRAEL,40,US,IS,IS,20150219,0,2015,ce0520d2e05f1683451c7bf456728576,-1,ce0520d2e05f1683451c7bf456728576
...,...,...,...,...,...,...,...,...,...,...,...,...
1202264,ISRAEL,POLAND,42,IS,UP,IS,20220317,2583,2022,e10c808f7da06b957263923b0f0ae15f,211,e10c808f7da06b957263923b0f0ae15f
1202265,POLAND,ISRAELI,43,PL,IS,IS,20220317,2583,2022,e10c808f7da06b957263923b0f0ae15f,211,e10c808f7da06b957263923b0f0ae15f
1202266,POLICE,REFUGEE,36,IS,IS,IS,20220317,2583,2022,be62976e9b5ba60fcdcda63ac409c0de,211,be62976e9b5ba60fcdcda63ac409c0de
1202267,POLAND,ISRAEL,43,PL,UP,IS,20220317,2583,2022,7d1276b99bca7ad94f1ed2062b8d67dd,-1,7d1276b99bca7ad94f1ed2062b8d67dd


In [102]:
# stats
filtered_results = filtered_ces_df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
filtered_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
filtered_results['nday_range'] = filtered_results['nday_max'] -filtered_results['nday_min'] + 1
print(filtered_results)
print(filtered_results.loc[0:, :].mean())
print(filtered_results.loc[0:, :].max())
print(filtered_results.loc[0:, :].min())

       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       628615      2583         0  1242.683421        2584
 0          306      1826      1799  1813.640523          28
 1          306      1830      1826  1828.637255           5
 2          306      1832      1830  1830.633987           3
 3          306      1834      1832  1833.032680           3
...         ...       ...       ...          ...         ...
 4531        48      2474      2444  2458.708333          31
 4532        31      1392      1359  1370.387097          34
 4533        34      1319      1286  1301.205882          34
 4534        19      1228      1215  1217.000000          14
 4535        36       793       781   784.694444          13

[4537 rows x 5 columns]
n_events       126.466931
nday_max      1288.450176
nday_min      1258.580026
nday_mean     1273.305292
nday_range      30.870150
dtype: float64
n_events       306.000000
nday_max   

In [103]:
filtered_ces_df.to_csv(path_or_buf='./GDELT_ce_raw.csv', sep='\t', index=False)

In [ ]:
# split ce into train valid and test

In [104]:
unique_time = filtered_ces_df['day'].unique()
tim2id = {name: idx for idx, name in enumerate(unique_time)}

In [105]:
tim2id

{20150219: 0,
 20150220: 1,
 20150221: 2,
 20150222: 3,
 20150223: 4,
 20150224: 5,
 20150225: 6,
 20150226: 7,
 20150227: 8,
 20150228: 9,
 20150301: 10,
 20150302: 11,
 20150303: 12,
 20150304: 13,
 20150305: 14,
 20150306: 15,
 20150307: 16,
 20150308: 17,
 20150309: 18,
 20150310: 19,
 20150311: 20,
 20150312: 21,
 20150313: 22,
 20150314: 23,
 20150315: 24,
 20150316: 25,
 20150317: 26,
 20150318: 27,
 20150319: 28,
 20150320: 29,
 20150321: 30,
 20150322: 31,
 20150323: 32,
 20150324: 33,
 20150325: 34,
 20150326: 35,
 20150327: 36,
 20150328: 37,
 20150329: 38,
 20150330: 39,
 20150331: 40,
 20150401: 41,
 20150402: 42,
 20150403: 43,
 20150404: 44,
 20150405: 45,
 20150406: 46,
 20150407: 47,
 20150408: 48,
 20150409: 49,
 20150410: 50,
 20150411: 51,
 20150412: 52,
 20150413: 53,
 20150414: 54,
 20150415: 55,
 20150416: 56,
 20150417: 57,
 20150418: 58,
 20150419: 59,
 20150420: 60,
 20150421: 61,
 20150422: 62,
 20150423: 63,
 20150424: 64,
 20150425: 65,
 20150426: 66,
 2015

In [106]:
test_max_date = filtered_ces_df['day'].max()
test_min_date = test_max_date-10000+1 # one year
val_max_date = test_min_date - 1
val_min_date = val_max_date-10000+1 # one year
train_max_date = val_min_date-1

In [107]:
train_max_date

20200317

In [108]:
# split ces
ceid2centertimid = {}
for ceid, valid_ce in enumerate(valid_ces):
    unique_timids = list(valid_ce['timid'].unique())
    timid_sum = 0
    for timid in unique_timids:
        timid_sum += timid * len(valid_ce[valid_ce['timid']==timid])
    centertimid = timid_sum*1.0/len(valid_ce)
    ceid2centertimid[ceid] = centertimid

In [109]:
ceid2centertimid

{0: 1813.640522875817,
 1: 1828.637254901961,
 2: 1830.6339869281046,
 3: 1833.0326797385621,
 4: 1835.7189542483661,
 5: 1838.5882352941176,
 6: 1840.7875816993464,
 7: 1842.7810457516339,
 8: 1845.0196078431372,
 9: 1847.264705882353,
 10: 1851.0424836601308,
 11: 1854.0718954248366,
 12: 1857.4738562091504,
 13: 1862.1568627450981,
 14: 1867.0947712418301,
 15: 1871.9346405228757,
 16: 1880.7810457516339,
 17: 1891.7908496732025,
 18: 1905.1568627450981,
 19: 1926.9411764705883,
 20: 1951.3692810457517,
 21: 1969.1503267973856,
 22: 1990.6827309236949,
 23: 229.8986928104575,
 24: 239.63398692810458,
 25: 251.7058823529412,
 26: 267.72549019607845,
 27: 284.31045751633985,
 28: 306.91830065359477,
 29: 319.062091503268,
 30: 327.2156862745098,
 31: 334.44117647058823,
 32: 345.94444444444446,
 33: 361.77777777777777,
 34: 383.2287581699346,
 35: 397.75,
 36: 2112.8529411764707,
 37: 2132.2549019607845,
 38: 2151.875816993464,
 39: 2167.3823529411766,
 40: 2185.4346405228757,
 41: 21

In [110]:
train_ceids, val_ceids, test_ceids = [], [], []
for ceid, centertimid in ceid2centertimid.items():
    if centertimid <= tim2id[train_max_date]:
        train_ceids.append(ceid)
    elif centertimid >= tim2id[val_min_date] and centertimid <= tim2id[val_max_date]:
        val_ceids.append(ceid)
    else:
        test_ceids.append(ceid)

In [111]:
train_ce_df = pd.concat([valid_ces[ceid] for ceid in train_ceids], ignore_index=True)
val_ce_df = pd.concat([valid_ces[ceid] for ceid in val_ceids], ignore_index=True)
test_ce_df = pd.concat([valid_ces[ceid] for ceid in test_ceids], ignore_index=True)

In [112]:
print(len(train_ceids))
print(len(val_ceids))
print(len(test_ceids))

3561
501
474


In [113]:
train_df = train_ce_df.sort_values(by=['day'], ignore_index=True)
val_df = val_ce_df.sort_values(by=['day'], ignore_index=True)
test_df = test_ce_df.sort_values(by=['day'], ignore_index=True)

In [114]:
# assign new ceids
old_ceids = list(train_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid
train_df['ce_id'] = [mapping[_] for _ in train_df['ce_id']]

In [115]:
old_ceids = list(val_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(train_df['ce_id']) + 1
val_df['ce_id'] = [mapping[_] for _ in val_df['ce_id']]

In [116]:
old_ceids = list(test_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(val_df['ce_id']) + 1
test_df['ce_id'] = [mapping[_] for _ in test_df['ce_id']]

In [117]:
all_df = pd.concat([train_df, val_df, test_df, new_outliers_dedu]).sort_values(by=['day'], ignore_index=True)

In [121]:
# stats
for df in [all_df]:
    print('Total docs')
    md5s = set()
    for md5_list in df['Md5_list']:
        md5s.update(md5_list.split(', '))
    print(len(md5s))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[0:, :].mean())
    print(df_results.loc[0:, :].max())
    print(df_results.loc[0:, :].min())

Total docs
273914
Total events
1202269
Number of entities
1557
Number of relations
239
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       628615      2583         0  1242.683421        2584
 0          306         4         0     1.663399           5
 1          306        11         0     6.232026          12
 2           97        29         0    14.443299          30
 3          163        47         0    18.564417          48
...         ...       ...       ...          ...         ...
 4531       116      2583      2574  2578.336207          10
 4532       306      2582      2576  2579.307190           7
 4533       306      2583      2578  2580.153595           6
 4534        65      2583      2581  2581.676923           3
 4535        83      2583      2582  2582.783133           2

[4537 rows x 5 columns]
n_events       126.466931
nday_max      1288.450176
nday_min      1258.580026
nday_mean     127

In [122]:
# stats
for df in [train_df, val_df, test_df, new_outliers_dedu]:
    print('Total docs')
    md5s = set()
    for md5_list in df['Md5_list']:
        md5s.update(md5_list.split(', '))
    print(len(md5s))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[:, :].mean())
    print(df_results.loc[:, :].max())
    print(df_results.loc[:, :].min())

Total docs
101487
Total events
441120
Number of entities
1546
Number of relations
233
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
0           306         4         0     1.663399           5
1           306        11         0     6.232026          12
2            97        29         0    14.443299          30
3           163        47         0    18.564417          48
4           188        41         0    18.414894          42
...         ...       ...       ...          ...         ...
3556        300      1856      1844  1850.223333          13
3557        306      1849      1846  1847.264706           4
3558         61      1853      1846  1848.852459           8
3559         51      1854      1847  1849.156863           8
3560        306      1853      1849  1851.042484           5

[3561 rows x 5 columns]
n_events       123.875316
nday_max      1029.642516
nday_min      1000.822803
nday_mean     1014

In [125]:
for df in [train_df, val_df, test_df, new_outliers_dedu]:
    print(df['day'].min())
    print(df['day'].max())
    print()

20150219
20200418

20200218
20210414

20200225
20220317

20150219
20220317



In [126]:
train_df.to_csv(path_or_buf='./GDELT_ce_train.csv', sep='\t', index=False)
val_df.to_csv(path_or_buf='./GDELT_ce_val.csv', sep='\t', index=False)
test_df.to_csv(path_or_buf='./GDELT_ce_test.csv', sep='\t', index=False)
all_df.to_csv(path_or_buf='./GDELT_ce.csv', sep='\t', index=False)

In [127]:
new_outliers_dedu = new_outliers_dedu.sort_values(by=['day'], ignore_index=True)
new_outliers_dedu.to_csv(path_or_buf='./GDELT_ce_outliers.csv', sep='\t', index=False)

In [ ]:
# filter unseen entity and relation after 20200317

In [128]:
anchor_df = all_df[all_df['day']<=20200317]
anchor_ent = (anchor_df['Actor1Name']._append(anchor_df['Actor2Name'])).unique()
anchor_rel = anchor_df['EventCode'].unique()

In [129]:
print(len(anchor_ent))
print(len(anchor_rel))

1555
239


In [130]:
seen_val_df = val_df[val_df['EventCode'].isin(anchor_rel)]
seen_test_df = test_df[test_df['EventCode'].isin(anchor_rel)]
seen_val_df = seen_val_df[(seen_val_df['Actor1Name'].isin(anchor_ent)) & (seen_val_df['Actor2Name'].isin(anchor_ent))]
seen_test_df = seen_test_df[(test_df['Actor1Name'].isin(anchor_ent)) & (seen_test_df['Actor2Name'].isin(anchor_ent))]

seen_outliers = all_df[all_df['ce_id']==-1]
seen_outliers = seen_outliers[seen_outliers['EventCode'].isin(anchor_rel)]
seen_outliers = seen_outliers[(seen_outliers['Actor1Name'].isin(anchor_ent)) & (seen_outliers['Actor2Name'].isin(anchor_ent))]

In [131]:
seen_df = pd.concat([train_df, seen_val_df, seen_test_df, seen_outliers]).sort_values(by=['day'], ignore_index=True)

In [132]:
# stats
for df in [seen_df]:
    print('Total docs')
    md5s = set()
    for md5_list in df['Md5_list']:
        md5s.update(md5_list.split(', '))
    print(len(md5s))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[0:, :].mean())
    print(df_results.loc[0:, :].max())
    print(df_results.loc[0:, :].min())

Total docs
273845
Total events
1201881
Number of entities
1555
Number of relations
239
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       628534      2583         0  1242.547589        2584
 0          306         4         0     1.663399           5
 1          306        11         0     6.232026          12
 2           97        29         0    14.443299          30
 3          163        47         0    18.564417          48
...         ...       ...       ...          ...         ...
 4531       116      2583      2574  2578.336207          10
 4532       306      2582      2576  2579.307190           7
 4533       306      2583      2578  2580.153595           6
 4534        65      2583      2581  2581.676923           3
 4535        83      2583      2582  2582.783133           2

[4537 rows x 5 columns]
n_events       126.399250
nday_max      1288.450176
nday_min      1258.581570
nday_mean     127

In [133]:
# stats
for df in [seen_val_df, seen_test_df, seen_outliers]:
    print('Total docs')
    md5s = set()
    for md5_list in df['Md5_list']:
        md5s.update(md5_list.split(', '))
    print(len(md5s))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[:, :].mean())
    print(df_results.loc[:, :].max())
    print(df_results.loc[:, :].min())

Total docs
14872
Total events
66794
Number of entities
1127
Number of relations
207
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
3561        196      1890      1825  1855.265306          66
3562        122      1904      1829  1876.795082          76
3563         18      1910      1833  1875.555556          78
3564         49      1884      1834  1863.285714          51
3565         54      1877      1839  1865.148148          39
...         ...       ...       ...          ...         ...
4057        164      2223      2204  2211.414634          20
4058         75      2246      2204  2216.573333          43
4059         54      2209      2206  2207.166667           4
4060        179      2219      2212  2213.837989           8
4061         29      2217      2215  2215.551724           3

[501 rows x 5 columns]
n_events       133.321357
nday_max      2058.862275
nday_min      2025.309381
nday_mean     2041.92

In [134]:
seen_outliers = seen_outliers.sort_values(by=['day'], ignore_index=True)

In [135]:
seen_val_df.to_csv(path_or_buf='./GDELT_ce_val_seen.csv', sep='\t', index=False)
seen_test_df.to_csv(path_or_buf='./GDELT_ce_test_seen.csv', sep='\t', index=False)
seen_outliers.to_csv(path_or_buf='./GDELT_ce_outliers_seen.csv', sep='\t', index=False)
seen_df.to_csv(path_or_buf='./GDELT_ce_seen.csv', sep='\t', index=False)

In [136]:
for df in [train_df, seen_val_df, seen_test_df, seen_outliers]:
    print(df['day'].min())
    print(df['day'].max())
    print()

20150219
20200418

20200218
20210414

20200225
20220317

20150219
20220317



In [ ]:
# filter by min day range and min ce size

In [137]:
ceid2size = {}
ces = list(seen_val_df['ce_id'].unique())
for ce in ces:
    ceid2size[ce] = len(seen_val_df[seen_val_df['ce_id']==ce])
    if len(seen_val_df[seen_val_df['ce_id']==ce]['timid'].unique()) == 1:
        print(ce)

In [138]:
for ce, size in ceid2size.items():
    if size < 10:
        print(ce)

4032


In [139]:
seen_val_df[seen_val_df['ce_id'].isin([4032])]

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
59651,AMIT,ISRAEL,10,IS,IS,IS,20210214,2187,2021,0e4123eb6103b71556b65f3ea3b3e725,4032,0e4123eb6103b71556b65f3ea3b3e725
59995,AMIT,ISRAEL,14,IS,IS,IS,20210216,2189,2021,8893a0ab0af049fcad6fbadc9fd32d82,4032,8893a0ab0af049fcad6fbadc9fd32d82
62219,PALESTINIAN,JEWISH,120,US,IS,IS,20210224,2197,2021,9a03ab07e800b59fb9ca83baaa4e1688,4032,9a03ab07e800b59fb9ca83baaa4e1688
62220,TERRORIST,ISRAELI,51,IS,IS,IS,20210224,2197,2021,9a03ab07e800b59fb9ca83baaa4e1688,4032,9a03ab07e800b59fb9ca83baaa4e1688
62221,EMPLOYEE,WEBSITE,90,IS,IS,IS,20210224,2197,2021,9a03ab07e800b59fb9ca83baaa4e1688,4032,9a03ab07e800b59fb9ca83baaa4e1688
62222,EMPLOYEE,MEDIA,90,IS,IS,IS,20210224,2197,2021,9a03ab07e800b59fb9ca83baaa4e1688,4032,9a03ab07e800b59fb9ca83baaa4e1688
62223,ISRAEL,JEWISH,120,IS,IS,IS,20210224,2197,2021,9a03ab07e800b59fb9ca83baaa4e1688,4032,9a03ab07e800b59fb9ca83baaa4e1688
63048,LAWMAKER,MEDIA,20,IS,IS,IS,20210227,2200,2021,2aee1d19977c7dc301e68de449d4f509,4032,2aee1d19977c7dc301e68de449d4f509
66334,REUTERS,ISRAEL,190,IZ,IS,IZ,20210319,2220,2021,c22024c98eab9cd409f7098133523611,4032,c22024c98eab9cd409f7098133523611


In [141]:
final_outliers = pd.concat([seen_outliers,
                            seen_val_df[seen_val_df['ce_id'].isin([4032])]])
final_outliers= final_outliers.sort_values(by=['day'], ignore_index=True)
final_outliers['ce_id'] = [-1] * len(final_outliers)

In [142]:
final_outliers

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,GAZA,ISRAEL,160,TU,IS,IS,20150219,0,2015,d6db17beecf1764d75e1b969d6e6e825,-1,d6db17beecf1764d75e1b969d6e6e825
1,EGYPT,LIBYA,195,QA,EG,EG,20150219,0,2015,aa867e4121b76fbc658672d8fc05ac01,-1,aa867e4121b76fbc658672d8fc05ac01
2,IRANIAN,SUNNI,190,IR,IZ,IZ,20150219,0,2015,2fa3a3f2a6ed935c8100cd014707438a,-1,2fa3a3f2a6ed935c8100cd014707438a
3,US GOVERNMENT,ISRAEL,20,IS,IS,IS,20150219,0,2015,c7583a7778d803b71cacb4b096278b9e,-1,c7583a7778d803b71cacb4b096278b9e
4,ISLAMIC,THE US,46,EG,QA,QA,20150219,0,2015,16870399ca4d9cf49252b06e533928ef,-1,16870399ca4d9cf49252b06e533928ef
...,...,...,...,...,...,...,...,...,...,...,...,...
628538,RUSSIAN,UKRAINIAN,62,IS,IS,IS,20220317,2583,2022,9d893b7061976befe206162d5f55c164,-1,9d893b7061976befe206162d5f55c164
628539,ISRAEL,LAWMAKER,53,IS,IS,UP,20220317,2583,2022,9fffa65c9dc78d724ce7e91a18ebdce6,-1,9fffa65c9dc78d724ce7e91a18ebdce6
628540,JOE BIDEN,TERRORIST,50,IR,US,US,20220317,2583,2022,798078bb6c12735fa387643057c48e8d,-1,798078bb6c12735fa387643057c48e8d
628541,IRAN,AUSTRIAN,42,IR,AU,AU,20220317,2583,2022,59ef34ee73cfbba2ed44611a19f2bfaf,-1,59ef34ee73cfbba2ed44611a19f2bfaf


In [144]:
fianl_val_df = seen_val_df.copy()

In [146]:
fianl_val_df = fianl_val_df[~fianl_val_df['ce_id'].isin([4032])]

In [148]:
old_ceids = list(fianl_val_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(train_df['ce_id']) + 1
fianl_val_df['ce_id'] = [mapping[_] for _ in fianl_val_df['ce_id']]

In [149]:
final_test_df = seen_test_df.copy()
old_ceids = list(final_test_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(fianl_val_df['ce_id']) + 1
final_test_df['ce_id'] = [mapping[_] for _ in final_test_df['ce_id']]

In [150]:
final_val_df = fianl_val_df.copy()

In [151]:
final_df = pd.concat([train_df, final_val_df, final_test_df, final_outliers]).sort_values(by=['day'], ignore_index=True)

In [152]:
final_df

,Actor1Name,Actor2Name,EventCode,Actor1ADM1Code,Actor2ADM1Code,EventADM1Code,day,timid,year,Md5,ce_id,Md5_list
0,IRAN,WENDY SHERMAN,46,IR,IR,US,20150219,0,2015,c109d1e16caa8aff4fced57a8c4045a0,0,c109d1e16caa8aff4fced57a8c4045a0
1,ISRAEL,IRAN,50,US,IS,US,20150219,0,2015,6d40d295cd67747ee9b56cc2c8099cfd,-1,6d40d295cd67747ee9b56cc2c8099cfd
2,ITALIAN,EGYPT,40,EG,EG,EG,20150219,0,2015,edf3db2729f3e81b20e446b94f7a6b6a,-1,edf3db2729f3e81b20e446b94f7a6b6a
3,THE US,CITIZEN,10,IS,IS,IS,20150219,0,2015,72dcb54c9b49830caefdad4bb7453d3c,-1,72dcb54c9b49830caefdad4bb7453d3c
4,KINGDOM,EGYPT,57,IZ,IZ,EG,20150219,0,2015,fea4f048e58509eb5ce3a95a356d3f3f,-1,fea4f048e58509eb5ce3a95a356d3f3f
...,...,...,...,...,...,...,...,...,...,...,...,...
1201876,UKRAINIAN,ISRAEL,51,UP,IS,IS,20220317,2583,2022,87a046d58f0a05e2c9c1bb9bd826b594,4532,87a046d58f0a05e2c9c1bb9bd826b594
1201877,UKRAINE,ISRAEL,42,UP,IS,UP,20220317,2583,2022,7b4bab0e38720b867a170944cf233c9b,4532,7b4bab0e38720b867a170944cf233c9b
1201878,UKRAINE,BRITISH,84,IR,IR,IR,20220317,2583,2022,3f48fb321a6a8e00f5f8a2c1fd756b87,4479,3f48fb321a6a8e00f5f8a2c1fd756b87
1201879,IRAN,BRITAIN,841,IR,IR,IR,20220317,2583,2022,079db1b369ae3926f797e3e88e072877,-1,079db1b369ae3926f797e3e88e072877


In [153]:
# stats
for df in [final_df]:
    print('Total docs')
    md5s = set()
    for md5_list in df['Md5_list']:
        md5s.update(md5_list.split(', '))
    print(len(md5s))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[0:, :].mean())
    print(df_results.loc[0:, :].max())
    print(df_results.loc[0:, :].min())

Total docs
273845
Total events
1201881
Number of entities
1555
Number of relations
239
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       628543      2583         0  1242.561268        2584
 0          306         4         0     1.663399           5
 1          306        11         0     6.232026          12
 2           97        29         0    14.443299          30
 3          163        47         0    18.564417          48
...         ...       ...       ...          ...         ...
 4530       116      2583      2574  2578.336207          10
 4531       306      2582      2576  2579.307190           7
 4532       306      2583      2578  2580.153595           6
 4533        65      2583      2581  2581.676923           3
 4534        83      2583      2582  2582.783133           2

[4536 rows x 5 columns]
n_events       126.425138
nday_max      1288.244763
nday_min      1258.376847
nday_mean     127

In [154]:
# stats
for df in [final_val_df, final_test_df, final_outliers]:
    print('Total docs')
    md5s = set()
    for md5_list in df['Md5_list']:
        md5s.update(md5_list.split(', '))
    print(len(md5s))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[:, :].mean())
    print(df_results.loc[:, :].max())
    print(df_results.loc[:, :].min())

Total docs
14867
Total events
66785
Number of entities
1127
Number of relations
207
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
3561        196      1890      1825  1855.265306          66
3562        122      1904      1829  1876.795082          76
3563         18      1910      1833  1875.555556          78
3564         49      1884      1834  1863.285714          51
3565         54      1877      1839  1865.148148          39
...         ...       ...       ...          ...         ...
4056        164      2223      2204  2211.414634          20
4057         75      2246      2204  2216.573333          43
4058         54      2209      2206  2207.166667           4
4059        179      2219      2212  2213.837989           8
4060         29      2217      2215  2215.551724           3

[500 rows x 5 columns]
n_events       133.570000
nday_max      2058.540000
nday_min      2024.986000
nday_mean     2041.61

In [155]:
final_val_df.to_csv(path_or_buf='./GDELT_ce_val_final.csv', sep='\t', index=False)
final_test_df.to_csv(path_or_buf='./GDELT_ce_test_final.csv', sep='\t', index=False)
final_outliers.to_csv(path_or_buf='./GDELT_ce_outliers_final.csv', sep='\t', index=False)
final_df.to_csv(path_or_buf='./GDELT_ce_final.csv', sep='\t', index=False)

In [156]:
for df in [train_df, final_val_df, final_test_df, final_outliers]:
    print(df['day'].min())
    print(df['day'].max())
    print()

20150219
20200418

20200218
20210414

20200225
20220317

20150219
20220317

